In [ ]:
# 1-1. Colab에 필요한 라이브러리 설치
!pip install -q ultralytics opencv-python-headless numpy cython_bbox loguru lap

# 1-2. ByteTrack 코드 클론 (필요 시)
!git clone https://github.com/ifzhang/ByteTrack.git
!pip install -q -r ByteTrack/requirements.txt

fatal: destination path 'ByteTrack' already exists and is not an empty directory.
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement onnxruntime==1.8.0 (from versions: 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.17.0, 1.17.1, 1.17.3, 1.18.0, 1.18.1, 1.19.0, 1.19.2, 1.20.0, 1.20.1, 1.21.0, 1.21.1)
ERROR: No matching distribution found for onnxruntime==1.8.0


In [ ]:
import sys
# ByteTrack 경로를 파이썬 모듈 검색 경로에 추가
sys.path.append('/content/ByteTrack')

In [ ]:
import cv2
import numpy as np
import csv
import os
from types import SimpleNamespace
from ultralytics import YOLO
from yolox.tracker.byte_tracker import BYTETracker

# -- 설정 --
INPUT_VIDEO     = "/content/60프레임.mp4"
YOLO_WEIGHTS    = "/content/best (3).pt"
TRACKED_OUTPUT  = "/content/output_4fps_tracked.mp4"
RAW_OUTPUT      = "/content/output_4fps_raw.mp4"   # ★ 원본 5FPS 영상
CSV_OUTPUT      = "/content/tracking_coords.csv"
DETECTED_DIR    = "/content/detected_frames"

# 임계값 및 파라미터
CONF_THRESH  = 0.1    # 낮춰서 검출 보장
TRACK_BUFFER = 50     # lost 상태 유지 프레임 수
MATCH_THRESH = 0.1    # IoU 매칭 완화
FPS_TARGET   = 4      # 타깃 FPS

# NumPy float alias (ByteTracker 호환)
np.float = float

# 출력 디렉토리 준비
os.makedirs(DETECTED_DIR, exist_ok=True)

# -- 모델 및 트래커 초기화 --
model = YOLO(YOLO_WEIGHTS)
args  = SimpleNamespace(
    track_thresh=CONF_THRESH,
    track_buffer=TRACK_BUFFER,
    match_thresh=MATCH_THRESH,
    mot20=False
)
tracker = BYTETracker(args, frame_rate=FPS_TARGET)

# -- CSV 파일 열기 및 헤더 작성 --
with open(CSV_OUTPUT, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["frame", "track_id", "x1", "y1", "x2", "y2", "center_x", "center_y"])

    # -- 비디오 입출력 설정 --
    cap      = cv2.VideoCapture(INPUT_VIDEO)
    orig_fps = cap.get(cv2.CAP_PROP_FPS)
    width    = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    interval = max(int(orig_fps // FPS_TARGET), 1)
    fourcc   = cv2.VideoWriter_fourcc(*"mp4v")

    # 트래킹된 영상용 Writer
    out_tracked = cv2.VideoWriter(TRACKED_OUTPUT, fourcc, FPS_TARGET, (width, height))
    # 원본 5FPS 영상용 Writer
    out_raw     = cv2.VideoWriter(RAW_OUTPUT,     fourcc, FPS_TARGET, (width, height))

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 5FPS에 맞춰 프레임 샘플링
        if frame_idx % interval == 0:
            # -- 1) 원본 프레임(샘플링된) 저장
            out_raw.write(frame)

            # -- 2) YOLO 검출 & ByteTrack 적용 --
            results = model.predict(source=frame, conf=CONF_THRESH, verbose=False)
            dets = []
            for box in results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                score         = float(box.conf[0].cpu())
                dets.append([x1, y1, x2 - x1, y2 - y1, score])

            dets_array = np.array(dets, dtype=np.float32) if dets else np.zeros((0, 5), dtype=np.float32)

            if dets_array.shape[0] > 0:
                online_targets = tracker.update(dets_array, (height, width), (height, width))

                for t in online_targets:
                    x1, y1, x2, y2 = map(int, t.tlbr)
                    track_id       = t.track_id
                    cx = (x1 + x2) / 2
                    cy = (y1 + y2) / 2

                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                    writer.writerow([frame_idx, track_id, x1, y1, x2, y2, cx, cy])

                # 트래킹된 프레임만 저장
                out_tracked.write(frame)
                img_path = os.path.join(DETECTED_DIR, f"frame_{frame_idx}.jpg")
                cv2.imwrite(img_path, frame)

        frame_idx += 1

    cap.release()
    out_tracked.release()
    out_raw.release()

print("✅ 원본 5FPS 영상과 트래킹 영상, 이미지, CSV가 모두 생성되었습니다.")

✅ 원본 5FPS 영상과 트래킹 영상, 이미지, CSV가 모두 생성되었습니다.


In [ ]:
import cv2
import numpy as np
import csv
# numpy float alias for ByteTracker compatibility
np.float = float
from types import SimpleNamespace
from ultralytics import YOLO
from yolox.tracker.byte_tracker import BYTETracker

# ===== 설정 =====
INPUT_VIDEO  = "/content/60프레임.mp4"
OUTPUT_VIDEO = "/content/output_2fps_tracked.mp4"
CSV_OUTPUT   = "/content/tracking_coords.csv"
YOLO_WEIGHTS = "/content/best (3).pt"
TARGET_FPS   = 2      # 최종 출력 FPS

# ===== 비디오 정보 읽기 =====
cap = cv2.VideoCapture(INPUT_VIDEO)
orig_fps = cap.get(cv2.CAP_PROP_FPS)
width    = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# 샘플링 간격 계산 (예: 60fps -> 2fps 이면 30)
interval = max(int(orig_fps // TARGET_FPS), 1)

# ===== 모델 및 트래커 초기화 =====
model = YOLO(YOLO_WEIGHTS)
tracker_args = SimpleNamespace(
    track_thresh=0.1,    # 낮춘 탐지 임계값
    track_buffer=50,     # lost 상태 유지 프레임 수
    match_thresh=0.1,    # 낮춘 IoU 매칭 임계값
    mot20=False
)
# ByteTracker에 원본 FPS를 전달
tracker = BYTETracker(tracker_args, frame_rate=int(orig_fps))

# ===== 출력 비디오 설정 =====
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out   = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, TARGET_FPS, (width, height))

# ===== CSV 파일 준비 =====
csv_file = open(CSV_OUTPUT, mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow([
    "orig_frame", "new_frame", "time_sec",
    "track_id", "x1", "y1", "x2", "y2",
    "center_x", "center_y"
])

# ===== 프레임별 탐지/트래킹 & 샘플링 =====
orig_frame = 0
new_frame  = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO로 탐지 수행
    results = model.predict(source=frame, conf=0.1, verbose=False)
    dets = []
    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        score = float(box.conf[0].cpu())
        dets.append([x1, y1, x2 - x1, y2 - y1, score])

    dets_array = np.array(dets, dtype=np.float32) if dets else np.zeros((0, 5), dtype=np.float32)

    # ByteTracker 업데이트
    online_targets = tracker.update(
        dets_array,
        (height, width),
        (height, width)
    )

    # 샘플링된 프레임만 저장/기록
    if orig_frame % interval == 0:
        time_sec = orig_frame / orig_fps
        for t in online_targets:
            x1, y1, x2, y2 = map(int, t.tlbr)
            tid = t.track_id
            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2
            csv_writer.writerow([
                orig_frame, new_frame, time_sec,
                tid, x1, y1, x2, y2, cx, cy
            ])
        out.write(frame)
        new_frame += 1

    orig_frame += 1

# ===== 정리 =====
cap.release()
out.release()
csv_file.close()
print(f"✅ 완료! {OUTPUT_VIDEO} 및 {CSV_OUTPUT} 생성됨 (샘플링 간격: {interval}).")


✅ 완료! /content/output_2fps_tracked.mp4 및 /content/tracking_coords.csv 생성됨 (샘플링 간격: 12).
